In [108]:
import gradio as gr
import ollama
import re

In [126]:
# 1. 게임 상태 초기화
game_state = {
    "loop": 0,
    "memory": [],
    "correct_count": 0,
    "max_correct": 15,
    "game_over": False
}

def reset_game_state():
    game_state["loop"] = 0
    game_state["memory"].clear()
    game_state["correct_count"] = 0
    game_state["game_over"] = False

In [127]:
# 2. 장면 생성 함수
def generate_scene(memory, loop_count):
    if loop_count == 0:
        return "당신은 전쟁터에서 깨어났습니다. 먼 곳에서 포성이 들리고, 먼지와 연기가 자욱한 하늘 아래에 처해 있습니다. 첫 임무를 앞두고 상황을 파악해야 합니다."

    memory_text = "\n".join([f"- '{m[0]}' → {m[1]}" for m in memory[-5:]]) if memory else "기억 없음"

    system_prompt = """
당신은 영화 '엣지 오브 투모로우'처럼 전장을 반복하는 병사입니다.

규칙:
- AI는 적의 현재 전황과 상황 설명만 생성합니다.
- 선택지는 간단한 명령어로 아래에 나열되며, 내용은 생략합니다.
- 선택지는 반드시 다음 형식으로 끝내세요:
선택지는 다음과 같습니다:
1. 돌진한다
2. 후퇴한다
3. 지원을 요청한다
"""

    user_prompt = f"""[루프 {loop_count}회차]
당신은 이전 루프에서 다음과 같은 행동을 했고, 그 결과를 모두 기억합니다:
{memory_text}

당신은 지금 어떤 상황에 처해 있으며, 어떤 전략적 판단이 필요한지 알려주세요.
(선택지는 자동으로 추출되며 본문에는 포함하지 마세요)"""

    response = ollama.chat(
        model="EEVE-Korean-10.8B",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response['message']['content'].strip()


In [128]:
# 3. 선택지 추출 함수 (간단한 선택지만)
def extract_choices_from_response(response):
    pattern = r"선택지는 다음과 같습니다:\s*1\.\s*(.*?)\s*2\.\s*(.*?)\s*3\.\s*(.*?)\s*$"
    match = re.search(pattern, response, re.DOTALL)
    if not match:
        return ["돌진한다", "후퇴한다", "지원 요청"]
    return [match.group(1).strip(), match.group(2).strip(), match.group(3).strip()]


In [129]:
# 4. 이벤트 처리 + 엔딩 체크
def apply_event(choice, response):
    if game_state["game_over"]:
        return "🎉 게임은 이미 종료되었습니다. 새로 시작하려면 재실행하세요."

    if any(word in response for word in ["죽", "사망", "전사", "끝났습니다"]):
        game_state["memory"].append((choice, "죽음"))
        if game_state["loop"] == 0:
            game_state["loop"] = 1
        else:
            game_state["loop"] += 1
        return f"☠️ 당신은 죽었습니다. 루프가 다시 시작됩니다.\n📜 기억: '{choice}' → 죽음"

    game_state["memory"].append((choice, "생존"))
    game_state["correct_count"] += 1

    if game_state["correct_count"] >= game_state["max_correct"]:
        game_state["game_over"] = True
        boss_prompt = f"""
[루프 {game_state['loop']}회차 – 최종 기억 통합 완료]
당신은 지금까지 {game_state['correct_count']}번의 올바른 선택을 통해 모든 기억을 되찾았습니다.
이제 최종 보스를 마주합니다. 어떻게 승리합니까?
"""
        boss_response = ollama.chat(
            model="EEVE-Korean-10.8B",
            messages=[{"role": "user", "content": boss_prompt}]
        )
        return f"👑 최종 보스 처치!\n{boss_response['message']['content'].strip()}"

    game_state["loop"] += 1
    return f"[루프 {game_state['loop']}회차 시작]\n✅ 생존했습니다! (누적 생존 {game_state['correct_count']}회)\n📜 기억: '{choice}' → 생존"


In [130]:
# 5. Gradio 인터페이스
with gr.Blocks() as demo:
    output_box = gr.Textbox(label="🧠 전장의 상황", lines=16, interactive=False)
    choice_radio = gr.Radio(choices=[], label="무엇을 하시겠습니까?", type="index", interactive=True)
    submit_btn = gr.Button("결정!")
    init_btn = gr.Button("🔄 게임 다시 시작하기")

    def on_decide(choice):
        response = generate_scene(game_state["memory"], game_state["loop"])
        choices = extract_choices_from_response(response)
        result = apply_event(choices[choice], response)

        if game_state["game_over"]:
            return f"{response}\n\n{result}", gr.update(choices=["🎉 게임 종료됨"], interactive=False)

        description = response.split("선택지는 다음과 같습니다:")[0].strip()  # ✅ 선택지는 제외하고 상황만 출력
        return f"{description}\n\n{result}", gr.update(choices=choices, value=None)

    def initialize():
        reset_game_state()
        first_response = generate_scene([], 0)
        initial_choices = extract_choices_from_response(first_response)
        description = first_response.split("선택지는 다음과 같습니다:")[0].strip()
        return description, gr.update(choices=initial_choices, value=None, interactive=True)

    init_btn.click(initialize, inputs=[], outputs=[output_box, choice_radio])
    submit_btn.click(on_decide, inputs=choice_radio, outputs=[output_box, choice_radio])

    demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7888
* Running on public URL: https://5595b64649b43c781d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2136, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1662, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 2106, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", 